## Finetune a pretrained model for sentence classifier

In [4]:
!pip install --upgrade torch torchvision --index-url https://download.pytorch.org/whl/cu118
!pip install Transformers==4.44.0
!pip install cloud-tpu-client torch_xla

Looking in indexes: https://download.pytorch.org/whl/cu118


In [1]:
import pandas as pd
import torch
from transformers import DistilBertTokenizerFast
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

In [2]:
label_dict = {
    'company': 0,
    'profile': 1,
    'tasks': 2,
    'benefits': 3
}

In [3]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=4)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
data_folder = "/content/drive/MyDrive/Colab Notebooks/data/"
train_data_path = data_folder + "train_set.csv"
test_data_path = data_folder + "test_set.csv"

In [5]:
df_train = pd.read_csv(train_data_path)
df_test = pd.read_csv(test_data_path)

In [6]:
train_texts, train_labels = list(df_train["sentence"]), list(df_train["label"].map(label_dict))
test_texts, test_labels = list(df_test["sentence"]), list(df_test["label"].map(label_dict))

In [7]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [8]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [9]:
train_dataset = CustomDataset(train_encodings, train_labels)
test_dataset = CustomDataset(test_encodings, test_labels)

In [10]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset             # evaluation dataset
)

In [11]:
trainer.train()

Step,Training Loss
10,1.377200
20,1.373100
30,1.351800
40,1.354400
50,1.324700
60,1.302000
70,1.216100
80,1.185100
90,1.124500
100,1.064400


TrainOutput(global_step=2961, training_loss=0.33960974259740473, metrics={'train_runtime': 1121.5492, 'train_samples_per_second': 42.239, 'train_steps_per_second': 2.64, 'total_flos': 3235857230434752.0, 'train_loss': 0.33960974259740473, 'epoch': 3.0})

Ref: https://huggingface.co/transformers/v3.2.0/custom_datasets.html#fine-tuning-with-custom-datasets